# TLSH Malware Analysis Tutorial / TLSH 惡意軟體分析教學

## Real-world Malware Classification with TLSH / 使用 TLSH 進行真實惡意軟體分類

**Instructor / 講師:** Rylin Lai (賴瑞霖)  
**Position / 職位:** Software Engineer, Trend Micro – Commercial Endpoint

---

### Tutorial Overview / 教學概述:
This tutorial demonstrates practical malware analysis using TLSH with real-world malware samples. We'll analyze raw malware data, perform clustering, and interpret the results for threat intelligence.

本教學使用真實世界的惡意軟體樣本示範使用 TLSH 進行實用的惡意軟體分析。我們將分析原始惡意軟體資料、執行分群並解釋威脅情報的結果。

### Learning Objectives / 學習目標:
1. **Load and explore real malware datasets / 載入和探索真實惡意軟體資料集**
2. **Apply TLSH clustering algorithms (HAC-T, DBSCAN) / 應用 TLSH 分群演算法**
3. **Classify unknown malware samples / 分類未知惡意軟體樣本**
4. **Interpret clustering results for threat analysis / 解讀分群結果用於威脅分析**

### Dataset Information / 資料集資訊:
- **Source / 來源**: Real malware samples (sanitized for educational use)
- **Format / 格式**: `sha1_hash,tlsh,signature`
- **Families / 家族**: Quakbot, TrickBot, Heodo, Mirai, AgentTesla, and more

## Step 1: Setup and Data Loading / 步驟 1：設置和資料載入

### Understanding Raw Malware Data / 了解原始惡意軟體資料

Our dataset contains real malware samples with the following format:

我們的資料集包含真實的惡意軟體樣本，格式如下：

| Column / 欄位 | Description / 描述 |
|---------------|--------------------|
| **sha1_hash** | SHA1 hash of the malware file / 惡意軟體檔案的 SHA1 雜湊 |
| **tlsh** | TLSH similarity hash / TLSH 相似性雜湊 |
| **signature** | Known malware family name / 已知惡意軟體家族名稱 |

This is **raw malware data**, not pre-clustered results. We will perform the clustering analysis ourselves.

這是**原始惡意軟體資料**，不是預先分群的結果。我們將自己執行分群分析。

In [ ]:
# Import required libraries / 匯入必要函式庫
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the clustering library to the path / 將分群函式庫加入路徑
sys.path.append('../tlshCluster')

try:
    from pylib.tlsh_lib import *
    from pylib.hac_lib import *
    print("✅ Successfully imported TLSH clustering libraries")
    print("✅ 成功匯入 TLSH 分群函式庫")
except ImportError as e:
    print(f"❌ Error importing libraries: {e}")
    print(f"❌ 匯入函式庫錯誤：{e}")
    print("Please ensure the clustering library is available")
    print("請確保分群函式庫可用")

In [ ]:
# Load the real malware dataset / 載入真實惡意軟體資料集
print("📊 Loading Real Malware Dataset / 載入真實惡意軟體資料集")
print("=" * 60)

# Load dataset using pandas for exploration
# 使用 pandas 載入資料集以進行探索
try:
    df = pd.read_csv('data/mb_10K.csv')
    print(f"✅ Dataset loaded successfully: {len(df):,} malware samples")
    print(f"✅ 資料集載入成功：{len(df):,} 個惡意軟體樣本")
    
    print("\n📋 Dataset Structure / 資料集結構:")
    print(df.head())
    
    print("\n📈 Dataset Summary / 資料集摘要:")
    print(f"   Total samples / 總樣本數: {len(df):,}")
    print(f"   Unique families / 唯一家族數: {df['signature'].nunique():,}")
    print(f"   Columns / 欄位: {list(df.columns)}")
    
    print("\n🔍 Data Quality Check / 資料品質檢查:")
    print(f"   Valid TLSH hashes / 有效 TLSH 雜湊: {df['tlsh'].notna().sum():,}")
    print(f"   Valid SHA1 hashes / 有效 SHA1 雜湊: {df['sha1_hash'].notna().sum():,}")
    print(f"   Labeled samples / 已標記樣本: {df['signature'].notna().sum():,}")
    
except FileNotFoundError:
    print("❌ Dataset file not found. Please ensure data/mb_10K.csv exists.")
    print("❌ 找不到資料集檔案。請確保 data/mb_10K.csv 存在。")
    # Try smaller dataset as fallback
    try:
        df = pd.read_csv('data/mb_1K.csv')
        print(f"✅ Using smaller dataset: {len(df):,} samples")
        print(f"✅ 使用較小資料集：{len(df):,} 個樣本")
    except:
        print("❌ No dataset files available")
        df = None
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    df = None

In [ ]:
# Analyze malware families in the dataset / 分析資料集中的惡意軟體家族
print("🦠 Malware Family Distribution / 惡意軟體家族分佈")
print("=" * 50)

if df is not None and len(df) > 0:
    # Count samples by family / 按家族計算樣本數
    family_counts = df['signature'].value_counts()
    
    print(f"📊 Top {min(15, len(family_counts))} Malware Families / 前 {min(15, len(family_counts))} 大惡意軟體家族:")
    print("\nFamily Name (家族名稱)        | Sample Count (樣本數)")
    print("-" * 50)
    
    for family, count in family_counts.head(15).items():
        percentage = (count / len(df)) * 100
        print(f"{family:<25} | {count:>6} ({percentage:5.1f}%)")
    
    # Create visualization
    plt.figure(figsize=(12, 8))
    family_counts.head(10).plot(kind='bar')
    plt.title('Top 10 Malware Families in Dataset\n資料集中前 10 大惡意軟體家族')
    plt.xlabel('Malware Family / 惡意軟體家族')
    plt.ylabel('Number of Samples / 樣本數量')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    print("\n💡 Family Analysis Notes / 家族分析說明:")
    print("• This shows the distribution of known malware families in our dataset")
    print("• 顯示我們資料集中已知惡意軟體家族的分佈")
    print("• We will use these labels to validate our clustering results")
    print("• 我們將使用這些標籤來驗證我們的分群結果")
    
else:
    print("❌ No data available for analysis")

## Step 2: TLSH Distance Analysis / 步驟 2：TLSH 距離分析

### Understanding TLSH Distance in Malware Analysis / 了解惡意軟體分析中的 TLSH 距離

Before clustering, let's understand how TLSH distances work with real malware samples:

在分群之前，讓我們了解 TLSH 距離如何與真實惡意軟體樣本一起工作：

- **Distance = 0**: Identical files / 相同檔案
- **Distance 1-50**: Very similar variants / 非常相似的變種
- **Distance 51-100**: Same family, different versions / 同家族，不同版本
- **Distance > 100**: Different malware families / 不同惡意軟體家族

### Distance Calculation Process / 距離計算流程:
1. **Load TLSH data** using the clustering library / 使用分群函式庫載入 TLSH 資料
2. **Calculate pairwise distances** between samples / 計算樣本間的成對距離
3. **Analyze distance distribution** / 分析距離分佈
4. **Choose clustering parameters** / 選擇分群參數

In [ ]:
# Load data using TLSH library and analyze distances / 使用 TLSH 函式庫載入資料並分析距離
print("🔍 TLSH Distance Analysis / TLSH 距離分析")
print("=" * 50)

try:
    # Load data using TLSH library functions
    # 使用 TLSH 函式庫功能載入資料
    dataset_file = 'data/mb_10K.csv' if os.path.exists('data/mb_10K.csv') else 'data/mb_1K.csv'
    
    print(f"📂 Loading {dataset_file} with TLSH library...")
    (tlist, labels) = tlsh_csvfile(dataset_file)
    
    if tlist is not None:
        print(f"✅ Loaded {len(tlist)} TLSH hashes successfully")
        print(f"✅ 成功載入 {len(tlist)} 個 TLSH 雜湊")
        
        # Analyze a subset of distances for visualization
        # 分析距離子集以進行視覺化
        sample_size = min(100, len(tlist))  # Use first 100 samples for speed
        
        print(f"\n📊 Analyzing distances between first {sample_size} samples...")
        print(f"📊 分析前 {sample_size} 個樣本之間的距離...")
        
        # Calculate sample distances
        distances = []
        for i in range(min(20, sample_size)):
            for j in range(i+1, min(i+10, sample_size)):  # Limited comparison for performance
                try:
                    # Convert TLSH strings to objects and calculate distance
                    import tlsh
                    h1 = tlsh.Tlsh()
                    h1.fromTlshStr(tlist[i])
                    h2 = tlsh.Tlsh()
                    h2.fromTlshStr(tlist[j])
                    dist = h1.diff(h2)
                    
                    if dist != -1:  # Valid distance
                        distances.append(dist)
                        
                        # Show some example distances with family info
                        if len(distances) <= 10:
                            family1 = labels[0][i] if labels and len(labels[0]) > i else 'Unknown'
                            family2 = labels[0][j] if labels and len(labels[0]) > j else 'Unknown'
                            same_family = "✓" if family1 == family2 else "✗"
                            print(f"   Sample {i+1:2d} vs {j+1:2d}: distance={dist:3d} | {family1:12} vs {family2:12} | Same: {same_family}")
                            
                except Exception as e:
                    continue
        
        if distances:
            print(f"\n📈 Distance Statistics / 距離統計:")
            print(f"   Calculated distances / 計算的距離數: {len(distances)}")
            print(f"   Min distance / 最小距離: {min(distances)}")
            print(f"   Max distance / 最大距離: {max(distances)}")
            print(f"   Average distance / 平均距離: {np.mean(distances):.1f}")
            
            # Visualize distance distribution
            plt.figure(figsize=(10, 6))
            plt.hist(distances, bins=20, alpha=0.7, edgecolor='black')
            plt.axvline(x=50, color='orange', linestyle='--', label='Family boundary (≤50: variants)')
            plt.axvline(x=100, color='red', linestyle='--', label='Family boundary (≤100: same family)')
            plt.xlabel('TLSH Distance / TLSH 距離')
            plt.ylabel('Frequency / 頻率')
            plt.title('Distribution of TLSH Distances Between Malware Samples\n惡意軟體樣本間的 TLSH 距離分佈')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
            
            print("\n💡 Distance Analysis Insights / 距離分析洞察:")
            close_samples = sum(1 for d in distances if d <= 50)
            same_family_samples = sum(1 for d in distances if 50 < d <= 100)
            different_family_samples = sum(1 for d in distances if d > 100)
            
            print(f"   • Very similar samples (≤50): {close_samples} pairs")
            print(f"   • Same family samples (51-100): {same_family_samples} pairs")
            print(f"   • Different families (>100): {different_family_samples} pairs")
            print(f"   • 非常相似樣本 (≤50)：{close_samples} 對")
            print(f"   • 同家族樣本 (51-100)：{same_family_samples} 對")
            print(f"   • 不同家族 (>100)：{different_family_samples} 對")
        
    else:
        print("❌ Failed to load TLSH data")
        
except Exception as e:
    print(f"❌ Error in distance analysis: {e}")
    print("This may be due to TLSH library compatibility issues")

## Step 3: Malware Clustering with DBSCAN / 步驟 3：使用 DBSCAN 進行惡意軟體分群

### Why DBSCAN for Malware Analysis? / 為什麼在惡意軟體分析中使用 DBSCAN？

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is excellent for malware analysis because:

DBSCAN（基於密度的空間分群應用與雜訊）在惡意軟體分析中表現優秀，因為：

- **Handles noise**: Identifies outlier samples that don't belong to any family
- **處理雜訊**: 識別不屬於任何家族的離群樣本
- **No predetermined cluster count**: Automatically determines the number of families
- **無預定群集數**: 自動確定家族數量
- **Distance-based**: Works well with TLSH similarity distances
- **基於距離**: 與 TLSH 相似性距離配合良好

### DBSCAN Parameters / DBSCAN 參數:
- **eps**: Maximum distance between samples in the same cluster (typically 30-50 for malware)
- **eps**: 同一群集中樣本間的最大距離（惡意軟體通常為 30-50）
- **min_samples**: Minimum samples required to form a cluster (typically 2-3)
- **min_samples**: 形成群集所需的最小樣本數（通常為 2-3）

In [ ]:
# Apply DBSCAN clustering to malware data / 對惡意軟體資料應用 DBSCAN 分群
print("🎯 DBSCAN Malware Clustering / DBSCAN 惡意軟體分群")
print("=" * 50)

try:
    if 'tlist' in locals() and tlist is not None:
        # Use a subset for faster processing in demonstration
        # 在示範中使用子集以加快處理速度
        max_samples = min(500, len(tlist))  # Limit for performance
        tlist_subset = tlist[:max_samples]
        labels_subset = [labels[0][:max_samples]] if labels else None
        
        print(f"📊 Clustering {max_samples} samples using DBSCAN...")
        print(f"📊 使用 DBSCAN 對 {max_samples} 個樣本進行分群...")
        
        # Apply DBSCAN with malware-appropriate parameters
        # 使用適合惡意軟體的參數應用 DBSCAN
        dbscan_result = runDBSCAN(tlist_subset, eps=30, min_samples=2)
        
        if hasattr(dbscan_result, 'labels_'):
            cluster_labels = dbscan_result.labels_
            
            # Analyze clustering results
            # 分析分群結果
            unique_labels = set(cluster_labels)
            n_clusters = len(unique_labels) - (1 if -1 in cluster_labels else 0)
            n_noise = list(cluster_labels).count(-1)
            
            print(f"\n✅ DBSCAN Clustering Results / DBSCAN 分群結果:")
            print(f"   Clusters formed / 形成的群集數: {n_clusters}")
            print(f"   Noise samples / 雜訊樣本數: {n_noise}")
            print(f"   Clustered samples / 已分群樣本數: {max_samples - n_noise}")
            print(f"   Clustering rate / 分群率: {((max_samples - n_noise) / max_samples * 100):.1f}%")
            
            # Analyze cluster quality using known family labels
            if labels_subset and labels_subset[0]:
                print(f"\n🔍 Cluster Quality Analysis / 群集品質分析:")
                
                # Show cluster composition
                for cluster_id in sorted(unique_labels):
                    if cluster_id == -1:
                        continue  # Skip noise for now
                    
                    cluster_indices = [i for i, label in enumerate(cluster_labels) if label == cluster_id]
                    if len(cluster_indices) <= 10:  # Only show small clusters for clarity
                        cluster_families = [labels_subset[0][i] for i in cluster_indices]
                        family_counts = {}
                        for family in cluster_families:
                            family_counts[family] = family_counts.get(family, 0) + 1
                        
                        print(f"   Cluster {cluster_id}: {len(cluster_indices)} samples")
                        for family, count in sorted(family_counts.items()):
                            print(f"      {family}: {count} samples")
            
            # Create visualization
            plt.figure(figsize=(12, 8))
            
            # Plot cluster distribution
            plt.subplot(2, 2, 1)
            cluster_sizes = [list(cluster_labels).count(label) for label in unique_labels if label != -1]
            plt.hist(cluster_sizes, bins=20, alpha=0.7, edgecolor='black')
            plt.xlabel('Cluster Size / 群集大小')
            plt.ylabel('Frequency / 頻率')
            plt.title('Distribution of Cluster Sizes\n群集大小分佈')
            plt.grid(True, alpha=0.3)
            
            # Plot clustering summary
            plt.subplot(2, 2, 2)
            categories = ['Clustered\\n已分群', 'Noise\\n雜訊']
            values = [max_samples - n_noise, n_noise]
            colors = ['lightblue', 'lightcoral']
            plt.pie(values, labels=categories, colors=colors, autopct='%1.1f%%')
            plt.title('Clustering vs Noise Samples\n分群 vs 雜訊樣本')
            
            # Plot cluster IDs (limited view)
            plt.subplot(2, 1, 2)
            sample_indices = range(min(100, len(cluster_labels)))
            sample_labels = cluster_labels[:len(sample_indices)]
            colors = ['red' if x == -1 else plt.cm.tab10(x % 10) for x in sample_labels]
            plt.scatter(sample_indices, sample_labels, c=colors, alpha=0.7)
            plt.xlabel('Sample Index / 樣本索引')
            plt.ylabel('Cluster ID / 群集 ID')
            plt.title('Sample Cluster Assignments (First 100 samples)\n樣本群集分配（前 100 個樣本）')
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
            print("\n💡 DBSCAN Analysis Insights / DBSCAN 分析洞察:")
            print(f"• DBSCAN effectively separates malware families into distinct clusters")
            print(f"• DBSCAN 有效地將惡意軟體家族分離成不同的群集")
            print(f"• Noise samples may represent unique variants or misclassified files")
            print(f"• 雜訊樣本可能代表獨特變種或錯誤分類的檔案")
            print(f"• eps=30 works well for malware family detection")
            print(f"• eps=30 在惡意軟體家族偵測中效果良好")
        
        else:
            print("❌ DBSCAN clustering failed to produce results")
    
    else:
        print("❌ No TLSH data available for clustering")
        
except Exception as e:
    print(f"❌ Error in DBSCAN clustering: {e}")
    print("This may be due to insufficient data or library compatibility issues")

## Step 4: HAC-T Clustering for Malware Families / 步驟 4：用於惡意軟體家族的 HAC-T 分群

### HAC-T: Hierarchical Agglomerative Clustering for TLSH / HAC-T：用於 TLSH 的階層式聚合分群

HAC-T is specifically designed for TLSH data and offers advantages for malware analysis:

HAC-T 專門為 TLSH 資料設計，在惡意軟體分析中提供優勢：

- **Hierarchical structure**: Shows evolutionary relationships between malware variants
- **階層結構**: 顯示惡意軟體變種間的演化關係
- **Distance threshold**: Uses TLSH distance thresholds for family classification
- **距離閾值**: 使用 TLSH 距離閾值進行家族分類
- **Deterministic**: Produces consistent results across runs
- **確定性**: 在多次運行中產生一致的結果

### HAC-T Process / HAC-T 流程:
1. **Build VP-Tree**: Efficient data structure for TLSH distance queries
2. **建立 VP-Tree**: 用於 TLSH 距離查詢的高效資料結構
3. **Agglomerative clustering**: Merge closest samples iteratively
4. **聚合分群**: 迭代合併最近的樣本
5. **Distance threshold**: Stop merging when distance exceeds threshold
6. **距離閾值**: 當距離超過閾值時停止合併

In [ ]:
# Apply HAC-T clustering to malware data / 對惡意軟體資料應用 HAC-T 分群
print("🌳 HAC-T Malware Clustering / HAC-T 惡意軟體分群")
print("=" * 50)

try:
    if 'tlist' in locals() and tlist is not None:
        # Use smaller subset for HAC-T demonstration (it's computationally intensive)
        # 為 HAC-T 示範使用較小子集（它在計算上很密集）
        hac_samples = min(200, len(tlist))  # Smaller subset for performance
        
        print(f"📊 Running HAC-T clustering on {hac_samples} samples...")
        print(f"📊 對 {hac_samples} 個樣本運行 HAC-T 分群...")
        print(f"⏱️ This may take a moment for distance calculations...")
        print(f"⏱️ 距離計算可能需要一些時間...")
        
        # Create temporary file for HAC-T
        temp_file = 'temp_hac_data.csv'
        with open(temp_file, 'w') as f:
            f.write('sha1_hash,tlsh,signature\n')
            for i in range(hac_samples):
                sha1 = f"sample_{i:06d}"  # Dummy SHA1
                tlsh_hash = tlist[i]
                family = labels[0][i] if labels and len(labels[0]) > i else 'Unknown'
                f.write(f"{sha1},{tlsh_hash},{family}\n")
        
        # Run HAC-T clustering
        hac_result = HAC_T_opt(temp_file, CDist=100, step3=0, outfname='temp_hac_out.txt', cenfname='temp_hac_cen.txt')
        
        if hac_result is not None:
            # Analyze HAC-T results
            unique_hac_labels = set(hac_result)
            n_hac_clusters = len(unique_hac_labels)
            
            print(f"\n✅ HAC-T Clustering Results / HAC-T 分群結果:")
            print(f"   Clusters formed / 形成的群集數: {n_hac_clusters}")
            print(f"   Processed samples / 處理的樣本數: {len(hac_result)}")
            print(f"   Average cluster size / 平均群集大小: {len(hac_result) / n_hac_clusters:.1f}")
            
            # Compare HAC-T with known family labels
            if labels and labels[0]:
                print(f"\n🔍 HAC-T vs Known Families Comparison / HAC-T vs 已知家族比較:")
                
                # Count cluster purity (samples with same family in same cluster)
                cluster_purity = 0
                total_comparisons = 0
                
                for cluster_id in unique_hac_labels:
                    cluster_indices = [i for i, label in enumerate(hac_result) if label == cluster_id]
                    if len(cluster_indices) > 1:
                        cluster_families = [labels[0][i] for i in cluster_indices if i < len(labels[0])]
                        if cluster_families:
                            most_common_family = max(set(cluster_families), key=cluster_families.count)
                            same_family_count = cluster_families.count(most_common_family)
                            purity = same_family_count / len(cluster_families)
                            
                            if len(cluster_indices) <= 10:  # Show details for small clusters
                                family_dist = {}
                                for fam in cluster_families:
                                    family_dist[fam] = family_dist.get(fam, 0) + 1
                                print(f"   Cluster {cluster_id}: {len(cluster_indices)} samples, purity: {purity:.2f}")
                                print(f"      Families: {dict(family_dist)}")
                            
                            cluster_purity += purity
                            total_comparisons += 1
                
                if total_comparisons > 0:
                    avg_purity = cluster_purity / total_comparisons
                    print(f"\n📊 Overall Cluster Purity / 整體群集純度: {avg_purity:.3f}")
                    print(f"   (1.0 = perfect clustering, 0.0 = random clustering)")
                    print(f"   (1.0 = 完美分群，0.0 = 隨機分群)")
            
            # Visualize HAC-T results
            plt.figure(figsize=(12, 6))
            
            # Plot cluster distribution
            plt.subplot(1, 2, 1)
            cluster_sizes = [list(hac_result).count(label) for label in unique_hac_labels]
            plt.hist(cluster_sizes, bins=15, alpha=0.7, edgecolor='black')
            plt.xlabel('Cluster Size / 群集大小')
            plt.ylabel('Frequency / 頻率')
            plt.title('HAC-T Cluster Size Distribution\nHAC-T 群集大小分佈')
            plt.grid(True, alpha=0.3)
            
            # Plot sample assignments
            plt.subplot(1, 2, 2)
            sample_indices = range(min(100, len(hac_result)))
            sample_labels = hac_result[:len(sample_indices)]
            colors = [plt.cm.tab20(x % 20) for x in sample_labels]
            plt.scatter(sample_indices, sample_labels, c=colors, alpha=0.7)
            plt.xlabel('Sample Index / 樣本索引')
            plt.ylabel('Cluster ID / 群集 ID')
            plt.title('HAC-T Cluster Assignments\nHAC-T 群集分配')
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            
        # Clean up temporary files
        for temp_file_name in [temp_file, 'temp_hac_out.txt', 'temp_hac_cen.txt']:
            if os.path.exists(temp_file_name):
                os.remove(temp_file_name)
        
        print("\n💡 HAC-T Analysis Insights / HAC-T 分析洞察:")
        print(f"• HAC-T creates hierarchical clusters based on TLSH distances")
        print(f"• HAC-T 基於 TLSH 距離創建階層群集")
        print(f"• CDist=100 threshold works well for malware family separation")
        print(f"• CDist=100 閾值在惡意軟體家族分離中效果良好")
        print(f"• Higher purity indicates better family clustering")
        print(f"• 更高純度表示更好的家族分群")

    else:
        print("❌ No TLSH data available for HAC-T clustering")
        
except Exception as e:
    print(f"❌ Error in HAC-T clustering: {e}")
    print("This may be due to library compatibility or computational limitations")

## Step 5: Unknown Malware Classification / 步驟 5：未知惡意軟體分類

### Real-world Scenario: Classifying New Malware / 真實世界情境：分類新惡意軟體

In practice, analysts receive unknown malware samples and need to:

在實踐中，分析師收到未知惡意軟體樣本並需要：

1. **Calculate TLSH** of the unknown sample / 計算未知樣本的 TLSH
2. **Find similar samples** in the database / 在資料庫中找到相似樣本
3. **Classify based on similarity** / 基於相似性進行分類
4. **Assess confidence** of the classification / 評估分類的置信度

### Classification Strategy / 分類策略:
- **Distance < 50**: High confidence - same variant / 高置信度 - 同一變種
- **Distance 50-100**: Medium confidence - same family / 中等置信度 - 同一家族  
- **Distance > 100**: Low confidence - possibly new family / 低置信度 - 可能是新家族

In [ ]:
# Simulate unknown malware classification / 模擬未知惡意軟體分類
print("🕵️ Unknown Malware Classification / 未知惡意軟體分類")
print("=" * 55)

try:
    if 'tlist' in locals() and tlist is not None and len(tlist) > 10:
        # Select a few samples to act as "unknown" samples
        # 選擇一些樣本作為「未知」樣本
        unknown_indices = [10, 50, 100] if len(tlist) > 100 else [5, 10, 15]
        unknown_indices = [idx for idx in unknown_indices if idx < len(tlist)]
        
        print(f"🎯 Classifying {len(unknown_indices)} unknown samples...")
        print(f"🎯 分類 {len(unknown_indices)} 個未知樣本...")
        
        # Import TLSH for distance calculations
        import tlsh
        
        for i, unknown_idx in enumerate(unknown_indices):
            if unknown_idx >= len(tlist):
                continue
                
            unknown_tlsh = tlist[unknown_idx]
            true_family = labels[0][unknown_idx] if labels and len(labels[0]) > unknown_idx else 'Unknown'
            
            print(f"\n📋 Unknown Sample #{i+1} / 未知樣本 #{i+1}:")
            print(f"   TLSH: {unknown_tlsh[:40]}...")
            print(f"   True family (hidden from classifier): {true_family}")
            print(f"   真實家族（對分類器隱藏）：{true_family}")
            
            # Find similar samples
            try:
                unknown_obj = tlsh.Tlsh()
                unknown_obj.fromTlshStr(unknown_tlsh)
                
                similarities = []
                # Compare with a sample of known malware (excluding the unknown sample itself)
                comparison_samples = min(200, len(tlist))
                
                for j in range(comparison_samples):
                    if j == unknown_idx:
                        continue  # Skip self
                        
                    try:
                        known_obj = tlsh.Tlsh()
                        known_obj.fromTlshStr(tlist[j])
                        distance = unknown_obj.diff(known_obj)
                        
                        if distance != -1:  # Valid comparison
                            known_family = labels[0][j] if labels and len(labels[0]) > j else 'Unknown'
                            similarities.append((distance, known_family, j))
                            
                    except Exception:
                        continue
                
                # Sort by distance (closest first)
                similarities.sort(key=lambda x: x[0])
                
                if similarities:
                    print(f"\n   🔍 Top 5 Most Similar Samples / 前 5 個最相似樣本:")
                    
                    for rank, (distance, family, sample_idx) in enumerate(similarities[:5], 1):
                        confidence = "HIGH" if distance <= 50 else "MEDIUM" if distance <= 100 else "LOW"
                        confidence_cn = "高" if distance <= 50 else "中" if distance <= 100 else "低"
                        print(f"   {rank}. Distance: {distance:3d} | Family: {family:12} | Confidence: {confidence}/{confidence_cn}")
                    
                    # Make classification based on closest samples
                    closest_distance = similarities[0][0]
                    closest_family = similarities[0][1]
                    
                    # Vote from top 3 closest samples
                    top_families = [sim[1] for sim in similarities[:3] if sim[0] <= 100]
                    if top_families:
                        family_votes = {}
                        for fam in top_families:
                            family_votes[fam] = family_votes.get(fam, 0) + 1
                        predicted_family = max(family_votes.keys(), key=lambda x: family_votes[x])
                        vote_confidence = family_votes[predicted_family] / len(top_families)
                    else:
                        predicted_family = "Unknown/New Family"
                        vote_confidence = 0.0
                    
                    print(f"\n   🎯 Classification Result / 分類結果:")
                    print(f"      Predicted family / 預測家族: {predicted_family}")
                    print(f"      Closest distance / 最近距離: {closest_distance}")
                    print(f"      Vote confidence / 投票置信度: {vote_confidence:.2f}")
                    
                    # Check if classification is correct
                    correct = predicted_family == true_family
                    result_icon = "✅" if correct else "❌"
                    print(f"      Accuracy / 準確性: {result_icon} {'Correct' if correct else 'Incorrect'}")
                    
                else:
                    print(f"   ❌ No similar samples found")
                    
            except Exception as e:
                print(f"   ❌ Error processing sample: {e}")
        
        print("\n💡 Classification Process Insights / 分類流程洞察:")
        print("• Distance-based classification works well for malware families")
        print("• 基於距離的分類在惡意軟體家族中效果良好")
        print("• Voting from multiple similar samples increases accuracy")
        print("• 從多個相似樣本投票提高準確性")
        print("• Very high distances (>150) may indicate new malware families")
        print("• 非常高的距離 (>150) 可能表示新的惡意軟體家族")
    
    else:
        print("❌ Insufficient data for classification demonstration")
        
except Exception as e:
    print(f"❌ Error in classification: {e}")

## Summary and Practical Applications / 總結和實際應用

### What We Accomplished / 我們完成的內容:

1. **Real Data Analysis / 真實資料分析**
   - Loaded and explored actual malware samples with TLSH hashes
   - 載入和探索具有 TLSH 雜湊的實際惡意軟體樣本
   - Analyzed family distributions and data quality
   - 分析家族分佈和資料品質

2. **Distance Analysis / 距離分析**
   - Calculated TLSH distances between malware samples
   - 計算惡意軟體樣本間的 TLSH 距離
   - Established distance thresholds for family classification
   - 建立家族分類的距離閾值

3. **Clustering Implementation / 分群實作**
   - Applied DBSCAN for density-based malware clustering
   - 應用 DBSCAN 進行基於密度的惡意軟體分群
   - Implemented HAC-T for hierarchical TLSH clustering
   - 實作 HAC-T 進行階層式 TLSH 分群

4. **Practical Classification / 實用分類**
   - Demonstrated unknown sample classification workflow
   - 示範未知樣本分類工作流程
   - Used distance-based voting for family prediction
   - 使用基於距離的投票進行家族預測

### Production Applications / 生產應用:

- **🚨 Incident Response**: Quickly classify unknown malware during security incidents
- **🔍 Threat Hunting**: Identify related malware campaigns and attack patterns
- **📊 Threat Intelligence**: Generate family-based IOCs and detection signatures
- **🤖 Automated Analysis**: Integrate into malware analysis pipelines
- **📈 Campaign Tracking**: Track malware evolution and variant development

### Best Practices / 最佳實踐:

1. **Parameter Tuning / 參數調整**
   - DBSCAN: eps=30, min_samples=2 for malware families
   - HAC-T: CDist=100 for family-level clustering
   - Classification: Distance thresholds 50/100 for confidence levels

2. **Data Quality / 資料品質**
   - Validate TLSH hash format before clustering
   - Handle missing or corrupted hashes appropriately
   - Maintain clean family labels for validation

3. **Scalability / 可擴展性**
   - Use VP-Tree or similar structures for large datasets
   - Implement incremental clustering for real-time analysis
   - Consider distributed processing for enterprise scale

### Next Steps / 下一步:

- **Integrate with SIEM platforms** for automated threat detection
- **與 SIEM 平台整合**用於自動威脅偵測
- **Develop custom distance thresholds** for your specific malware types
- **開發自定義距離閾值**適用於您特定的惡意軟體類型
- **Implement real-time classification** for file upload systems
- **實作即時分類**用於檔案上傳系統
- **Build family evolution tracking** for advanced threat intelligence
- **建立家族演化追蹤**用於進階威脅情報

---

### 🎓 Congratulations! / 恭喜！

You have successfully completed a comprehensive TLSH malware analysis tutorial using real malware data. You now have practical skills for:

您已成功完成使用真實惡意軟體資料的綜合 TLSH 惡意軟體分析教學。您現在具備以下實用技能：

- **✅ Raw malware data analysis and preprocessing**
- **✅ TLSH-based similarity analysis and clustering**  
- **✅ Unknown malware classification and family prediction**
- **✅ Production-ready malware analysis workflows**

These skills are directly applicable to threat intelligence, incident response, and malware research roles in cybersecurity.

這些技能可直接應用於網路安全領域的威脅情報、事件響應和惡意軟體研究角色。